In [1]:
import sys
sys.path.insert(0, "../")

In [2]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch
def success_score(inputs):
    with torch.inference_mode():
        outputs = model(**inputs)
    logits_per_text = outputs.logits_per_text
    logits_per_image = outputs.logits_per_image
    text_probs = logits_per_text.softmax(dim=1)
    image_probs = logits_per_image.softmax(dim=1)
    text_score = False
    img_score = False
    group_score = False 
    if (text_probs[0][0]>text_probs[1][0]) and (text_probs[1][1]>text_probs[0][1]):
        text_score = True
    if (image_probs[0][0]>image_probs[1][0]) and (image_probs[1][1]>image_probs[0][1]):
        img_score = True
    if text_score and img_score: 
        group_score = True
    return text_score, img_score, group_score

In [7]:
from source.wino_loader import extract_captions_winoground
winoground = extract_captions_winoground(dir = "../dataset/examples.jsonl")

from PIL import Image
hit_score = 0
hit_score_text =0 
hit_score_img = 0 
for i in range(len(winoground['caption_0'])):
    images =[]
    texts =[]
    img1 = "../dataset/images/"+ winoground["image_0"][i]+".png"
    img2 = "../dataset/images/"+ winoground["image_1"][i]+".png"
    images.append(Image.open(img1))
    images.append(Image.open(img2))
    texts. append(winoground["caption_0"][i])
    texts. append(winoground["caption_1"][i])
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    text_score, img_score, group_score =success_score(inputs)
    if group_score:
        hit_score +=1
    if text_score:
        hit_score_text +=1
    if img_score:
        hit_score_img +=1
    del images, texts


In [8]:
print(hit_score_text, hit_score_img, hit_score)
print(len(winoground['caption_0']))
print(f"Hit Rate (Percentage): {1 - hit_score/(len(winoground['caption_0']))}")

257 257 257
400
Hit Rate (Percentage): 0.35750000000000004


### THIS PORTION CONTAINS THE CODE TO COMPARE THE PERFORAMNCE OF CLIP MODEL WITH WINOGROUND AND MODIFIED WINOGROUND DATASET ###
*MODIFIED CAPTIONS = 50*

In [1]:
import sys
sys.path.insert(0, "../")


from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

n_samples=50

import torch
def success_score(inputs):
    with torch.inference_mode():
        outputs = model(**inputs)
    logits_per_text = outputs.logits_per_text
    logits_per_image = outputs.logits_per_image
    text_probs = logits_per_text.softmax(dim=1)
    image_probs = logits_per_image.softmax(dim=1)
    text_score = False
    img_score = False
    group_score = False 
    if (text_probs[0][0]>text_probs[1][0]) and (text_probs[1][1]>text_probs[0][1]):
        text_score = True
    if (image_probs[0][0]>image_probs[1][0]) and (image_probs[1][1]>image_probs[0][1]):
        img_score = True
    if text_score and img_score: 
        group_score = True
    return text_score, img_score, group_score



from source.wino_loader import extract_edited_captions_winoground
modified_winoground = extract_edited_captions_winoground(dir="../dataset/edited_captions.xls", edited_captions=n_samples)

from PIL import Image
wino_data_hit_score = 0
wino_data_hit_score_text =0 
wino_data_hit_score_img = 0 
modified_data_hit_score = 0
modified_data_hit_score_text =0 
modified_data_hit_score_img = 0
##----- compare wino and modified outcome-------
wino_0_modified_0 = 0
wino_0_modified_1 = 0
wino_1_modified_0 = 0
wino_1_modified_1 = 0
for i in range(len(modified_winoground['true_caption_0'])):
    images =[]
    texts =[]
    img1 = "../dataset/images/"+ modified_winoground["image_0"][i]+".png"
    img2 = "../dataset/images/"+ modified_winoground["image_1"][i]+".png"
    images.append(Image.open(img1))
    images.append(Image.open(img2))
    texts. append(modified_winoground["true_caption_0"][i])
    texts. append(modified_winoground["true_caption_1"][i])
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    wino_text_score, wino_img_score, wino_group_score =success_score(inputs)
    if wino_group_score:
        wino_data_hit_score +=1
    if wino_text_score:
        wino_data_hit_score_text +=1
    if wino_img_score:
        wino_data_hit_score_img +=1
    texts =[]
    texts. append(modified_winoground["modified_caption_0"][i])
    texts. append(modified_winoground["modified_caption_1"][i])
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    modified_text_score, modified_img_score, modified_group_score =success_score(inputs)
    if modified_group_score:
        modified_data_hit_score +=1
    if modified_text_score:
        modified_data_hit_score_text +=1
    if modified_img_score:
        modified_data_hit_score_img +=1


    if wino_group_score==False and modified_group_score==False:
        wino_0_modified_0 +=1
    elif wino_group_score==False and modified_group_score==True:
        wino_0_modified_1 +=1
    elif wino_group_score==True and modified_group_score==False:
        wino_1_modified_0 +=1
    elif wino_group_score==True and modified_group_score==True:
        wino_1_modified_1 +=1
    del images, texts

print(f"Hit Ratio for Winoground: {wino_data_hit_score/n_samples}")
print(f"Hit Ratio for Modified Winoground: {modified_data_hit_score/n_samples}")
print(f"Winoground Hit Score: {wino_data_hit_score}")
print(f"Modified Winoground Hit Score: {modified_data_hit_score}")
print(f"No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: {wino_0_modified_0}")
print(f"No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: {wino_0_modified_1}")
print(f"No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: {wino_1_modified_0}")
print(f"No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: {wino_1_modified_1}")

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hit Ratio for Winoground: 0.64
Hit Ratio for Modified Winoground: 0.76
Winoground Hit Score: 32
Modified Winoground Hit Score: 38
No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: 6
No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: 12
No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: 6
No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: 26


### THIS PORTION CONTAINS THE CODE TO COMPARE THE PERFORAMNCE OF CLIP MODEL WITH WINOGROUND AND MODIFIED WINOGROUND DATASET ###
*MODIFIED CAPTIONS = 100*

In [1]:
import sys
sys.path.insert(0, "../")


from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

n_samples=100

import torch
def success_score(inputs):
    with torch.inference_mode():
        outputs = model(**inputs)
    logits_per_text = outputs.logits_per_text
    logits_per_image = outputs.logits_per_image
    text_probs = logits_per_text.softmax(dim=1)
    image_probs = logits_per_image.softmax(dim=1)
    text_score = False
    img_score = False
    group_score = False 
    if (text_probs[0][0]>text_probs[1][0]) and (text_probs[1][1]>text_probs[0][1]):
        text_score = True
    if (image_probs[0][0]>image_probs[1][0]) and (image_probs[1][1]>image_probs[0][1]):
        img_score = True
    if text_score and img_score: 
        group_score = True
    return text_score, img_score, group_score



from source.wino_loader import extract_edited_captions_winoground
modified_winoground = extract_edited_captions_winoground(dir="../dataset/edited_captions.xls", edited_captions=n_samples)

from PIL import Image
wino_data_hit_score = 0
wino_data_hit_score_text =0 
wino_data_hit_score_img = 0 
modified_data_hit_score = 0
modified_data_hit_score_text =0 
modified_data_hit_score_img = 0
##----- compare wino and modified outcome-------
wino_0_modified_0 = 0
wino_0_modified_1 = 0
wino_1_modified_0 = 0
wino_1_modified_1 = 0
for i in range(len(modified_winoground['true_caption_0'])):
    images =[]
    texts =[]
    img1 = "../dataset/images/"+ modified_winoground["image_0"][i]+".png"
    img2 = "../dataset/images/"+ modified_winoground["image_1"][i]+".png"
    images.append(Image.open(img1))
    images.append(Image.open(img2))
    texts. append(modified_winoground["true_caption_0"][i])
    texts. append(modified_winoground["true_caption_1"][i])
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    wino_text_score, wino_img_score, wino_group_score =success_score(inputs)
    if wino_group_score:
        wino_data_hit_score +=1
    if wino_text_score:
        wino_data_hit_score_text +=1
    if wino_img_score:
        wino_data_hit_score_img +=1
    texts =[]
    texts. append(modified_winoground["modified_caption_0"][i])
    texts. append(modified_winoground["modified_caption_1"][i])
    inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
    modified_text_score, modified_img_score, modified_group_score =success_score(inputs)
    if modified_group_score:
        modified_data_hit_score +=1
    if modified_text_score:
        modified_data_hit_score_text +=1
    if modified_img_score:
        modified_data_hit_score_img +=1


    if wino_group_score==False and modified_group_score==False:
        wino_0_modified_0 +=1
    elif wino_group_score==False and modified_group_score==True:
        wino_0_modified_1 +=1
    elif wino_group_score==True and modified_group_score==False:
        wino_1_modified_0 +=1
    elif wino_group_score==True and modified_group_score==True:
        wino_1_modified_1 +=1
    del images, texts

print(f"Hit Ratio for Winoground: {wino_data_hit_score/n_samples}")
print(f"Hit Ratio for Modified Winoground: {modified_data_hit_score/n_samples}")
print(f"Winoground Hit Score: {wino_data_hit_score}")
print(f"Modified Winoground Hit Score: {modified_data_hit_score}")
print(f"No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: {wino_0_modified_0}")
print(f"No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: {wino_0_modified_1}")
print(f"No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: {wino_1_modified_0}")
print(f"No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: {wino_1_modified_1}")

c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hit Ratio for Winoground: 0.66
Hit Ratio for Modified Winoground: 0.82
Winoground Hit Score: 66
Modified Winoground Hit Score: 82
No. of Samples where BOTH the WINOGROUND captions and MODIFIED pairs are identified INCORRECTLY: 9
No. of Samples where the WINOGROUND captions are indentified INCORRECTLY but MODIFIED pairs are identified CORRECTLY: 25
No. of Samples where the WINOGROUND captions are indentified CORRECTLY but MODIFIED pairs are identified INCORRECTLY: 9
No. of Samples where both the WINOGROUND captions and MODIFIED pairs are identified CORRECTLY: 57


In [11]:
type(winoground)

dict

In [1]:
from PIL import Image
import requests
import torch 
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")



c:\Users\DELL\anaconda3\envs\py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image = [Image.open('../dataset/images/ex_0_img_0.png')]
inputs = processor(text=["an old person kisses a young person","a young person kisses an old person"], images=image, return_tensors="pt", padding=True)

with torch.inference_mode():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
logits_per_text = outputs.logits_per_text
print(logits_per_image.softmax(dim=1))
print("-------------------")
print(logits_per_text.softmax(dim=1))

tensor([[0.3177, 0.6823]])
-------------------
tensor([[1.],
        [1.]])


In [3]:
image = [Image.open('../dataset/images/ex_0_img_0.png')]
image = [Image.open('../dataset/images/ex_0_img_0.png'), Image.open('../dataset/images/ex_0_img_1.png')]
inputs = processor(text=["an old person kisses a young person","a young person kisses an old person"], images=image, return_tensors="pt", padding=True)
with torch.inference_mode():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
logits_per_text = outputs.logits_per_text
print(logits_per_image.softmax(dim=1))
print("-------------------")
print(logits_per_text.softmax(dim=1))

tensor([[0.3177, 0.6823],
        [0.2370, 0.7630]])
-------------------
tensor([[0.7697, 0.2303],
        [0.6904, 0.3096]])


In [4]:
image = [Image.open('../dataset/images/ex_0_img_0.png')]
image = [Image.open('../dataset/images/ex_0_img_0.png'), Image.open('../dataset/images/ex_0_img_1.png'), Image.open('../dataset/images/ex_1_img_0.png')]
inputs = processor(text=["an old person kisses a young person","a young person kisses an old person"], images=image, return_tensors="pt", padding=True)
with torch.inference_mode():
    outputs = model(**inputs)

logits_per_image = outputs.logits_per_image
logits_per_text = outputs.logits_per_text
print(logits_per_image.softmax(dim=1))
print("-------------------")
print(logits_per_text.softmax(dim=1))

tensor([[0.3177, 0.6823],
        [0.2370, 0.7630],
        [0.1470, 0.8530]])
-------------------
tensor([[7.6968e-01, 2.3027e-01, 5.1947e-05],
        [6.9031e-01, 3.0957e-01, 1.2595e-04]])
